In [5]:
import numpy as np

# read file 

data = open("Input/data.txt", "r")
param = open("Input/parameters.txt", "r")



In [6]:
# read lines from file as numbers   

rainfall = []
for line in data:
    rainfall.append(float(line))


N = int(param.readline())

trans_matrix = []

for i in range(N):
    # read line from param
    p = [float(x) for x in param.readline().split()]
    trans_matrix.append(p)


gauss_distro = []
for i in range(N):
    # read line from param
    p = [float(x) for x in param.readline().split()]
    gauss_distro.append(p)




In [7]:
# gaussian distribution function

def gauss(x, mu, sigma):
    return (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-((x-mu)**2)/(2*sigma**2))

    



In [ ]:

# emission probabilities
emisson_matrix = []

for r in rainfall:
    emit_prob = []
    for i in range(N):
        emit_prob.append(gauss(r, gauss_distro[i][0], gauss_distro[i][1]))
    emisson_matrix.append(emit_prob)





    
        

In [ ]:
# viterbi algorithm

def viterbi(trans_matrix,emission_matix,start) :
    